In [1]:
import cv2
import time
import dlib
import numpy as np
from imutils import face_utils

In [2]:
# The detector is used to detect face in the captured frame. Feel free to use your own model here
detector = dlib.get_frontal_face_detector()

# The predictor will output the face landmarks
predictor_68 = dlib.shape_predictor('./Models/shape_predictor_68_face_landmarks.dat')
predictor_5 = dlib.shape_predictor('./Models/shape_predictor_5_face_landmarks.dat')

In [3]:
# You can use path to video 
# or index of mounted camers 
# or use ip address(IPWebcam app) inplace of 0
cam = cv2.VideoCapture(0)

numbers_toggle = True
model_toggle = True

while True:
    _, frame = cam.read()# Frame size is 640x48
#     frame = cv2.resize(frame, (400,400))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0) #how many faces are available and their coordinates
    
    if len(rects) > 0:
        text = "{} face(s) found".format(len(rects))
        cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0, 0, 255), 2)
        
    for rect in rects:
# The rect object has 4 attributes left, top, right, bottom ==> rect_to_bb() ==> [x, y, width, height]
        (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
        cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH),(0, 255, 0), 1)

# face object which contain the landmark predictions
        if model_toggle == True:
            shape = predictor_68(gray, rect)
        else:
            shape = predictor_5(gray, rect)
        shape = face_utils.shape_to_np(shape)
# shape object contain attributes part(i).x and .part(i).y ==> shape_to_np() ==> ndarray[(xi, yi)]        

        for (i, (x, y)) in enumerate(shape):
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            if numbers_toggle == True:
                cv2.putText(frame, str(i + 1), (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    elif key == ord("n"):
        numbers_toggle = not numbers_toggle
    elif key == ord('m'):
        model_toggle = not model_toggle
    
cv2.destroyAllWindows()
cam.release()